In [1]:
import warnings
warnings.filterwarnings("ignore")

#######################################################################################
import matplotlib
import matplotlib.pyplot as plt
#matplotlib內建不支援中文，解決辦法：每次預先指定字體
matplotlib.rcParams.update({'font.size': 36})
matplotlib.use('qt4agg')
myfont = matplotlib.font_manager.FontProperties(fname='C:\\Windows\\Fonts\\msjh.ttc')
#指定字體
matplotlib.rcParams['axes.unicode_minus']=False

from matplotlib.ticker import FormatStrFormatter
majorFormatter = FormatStrFormatter('%0.f') #設定圖表浮點數的格式
%matplotlib inline 
#繪圖完就直接顯示該圖，省略每次繪圖完都要輸入plt.show指令的動作
#######################################################################################
import seaborn as sns


import numpy as np
# 設定array的float格式
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
import pandas as pd
pd.set_option('display.max_columns', 100)  #設定可顯示欄位的上限
pd.options.display.float_format = '{:,.6f}'.format  #設定浮點數的格式

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

Using TensorFlow backend.


In [3]:
data = pd.read_excel('rawdata/nextweek.xlsx', encoding='utf8')

In [4]:
data = data.fillna(0)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 93 columns):
account         1000 non-null object
num_20180301    1000 non-null int64
num_20180302    1000 non-null float64
num_20180303    1000 non-null float64
num_20180304    1000 non-null float64
num_20180305    1000 non-null float64
num_20180306    1000 non-null float64
num_20180307    1000 non-null float64
num_20180308    1000 non-null float64
num_20180309    1000 non-null float64
num_20180310    1000 non-null float64
num_20180311    1000 non-null float64
num_20180312    1000 non-null float64
num_20180313    1000 non-null float64
num_20180314    1000 non-null float64
num_20180315    1000 non-null float64
num_20180316    1000 non-null float64
num_20180317    1000 non-null float64
num_20180318    1000 non-null float64
num_20180319    1000 non-null float64
num_20180320    1000 non-null float64
num_20180321    1000 non-null float64
num_20180322    1000 non-null float64
num_20180323    1000 no

In [5]:
data.head(10)

,account,num_20180301,num_20180302,num_20180303,num_20180304,num_20180305,num_20180306,num_20180307,num_20180308,num_20180309,num_20180310,num_20180311,num_20180312,num_20180313,num_20180314,num_20180315,num_20180316,num_20180317,num_20180318,num_20180319,num_20180320,num_20180321,num_20180322,num_20180323,num_20180324,num_20180325,num_20180326,num_20180327,num_20180328,num_20180329,num_20180330,num_20180331,num_20180401,num_20180402,num_20180403,num_20180404,num_20180405,num_20180406,num_20180407,num_20180408,num_20180409,num_20180410,num_20180411,num_20180412,num_20180413,num_20180414,num_20180415,num_20180416,num_20180417,num_20180418,num_20180419,num_20180420,num_20180421,num_20180422,num_20180423,num_20180424,num_20180425,num_20180426,num_20180427,num_20180428,num_20180429,num_20180430,num_20180501,num_20180502,num_20180503,num_20180504,num_20180505,num_20180506,num_20180507,num_20180508,num_20180509,num_20180510,num_20180511,num_20180512,num_20180513,num_20180514,num_20180515,num_20180516,num_20180517,num_20180518,num_20180519,num_20180520,num_20180521,num_20180522,num_20180523,num_20180524,num_20180525,num_20180526,num_20180527,num_20180528,num_20180529,num_20180530,num_20180531
0,kiss9985,1,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
1,kitming20,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,kitty1100,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,kitty77688,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0

In [6]:
train_df = data.iloc[:,1:32].copy()
train_df.head(1)

,num_20180301,num_20180302,num_20180303,num_20180304,num_20180305,num_20180306,num_20180307,num_20180308,num_20180309,num_20180310,num_20180311,num_20180312,num_20180313,num_20180314,num_20180315,num_20180316,num_20180317,num_20180318,num_20180319,num_20180320,num_20180321,num_20180322,num_20180323,num_20180324,num_20180325,num_20180326,num_20180327,num_20180328,num_20180329,num_20180330,num_20180331
0,1,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000


In [7]:
test_df = data.iloc[:,32:62].copy()
test_df.head(1)

,num_20180401,num_20180402,num_20180403,num_20180404,num_20180405,num_20180406,num_20180407,num_20180408,num_20180409,num_20180410,num_20180411,num_20180412,num_20180413,num_20180414,num_20180415,num_20180416,num_20180417,num_20180418,num_20180419,num_20180420,num_20180421,num_20180422,num_20180423,num_20180424,num_20180425,num_20180426,num_20180427,num_20180428,num_20180429,num_20180430
0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000


In [8]:
#預測對象(y)，未來1週是否登入
train_y = data[['num_20180401', 'num_20180402','num_20180403','num_20180404','num_20180405','num_20180406','num_20180407']]
train_y.head()

,num_20180401,num_20180402,num_20180403,num_20180404,num_20180405,num_20180406,num_20180407
0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
#預測對象(y)，未來1週是否登入
test_y = data[['num_20180501', 'num_20180502','num_20180503','num_20180504','num_20180505','num_20180506','num_20180507']]
test_y.head()

,num_20180501,num_20180502,num_20180503,num_20180504,num_20180505,num_20180506,num_20180507
0,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
#由於LSTM需要輸入為3維，利用np.reshape轉換成3維資料
train_value = train_df.values
train_value = np.reshape(train_value, (train_value.shape[0], train_value.shape[1], 1))
print(train_value.shape) #樣本數, timesteps, 特徵數

(1000, 31, 1)


In [11]:
test_value = test_df.values
test_value = np.reshape(test_value, (test_value.shape[0], test_value.shape[1], 1))
print(test_value.shape) #樣本數, timesteps, 特徵數

(1000, 30, 1)


## 訓練
### 根據前一個月(20180301~20180331)預測未來7天(20180401~20180407)的登入次數

In [12]:
def model0():
    model = Sequential()
    model.add(LSTM(units=128,input_shape=(train_value.shape[1], train_value.shape[2]), return_sequences=True))#timestep, feature
    model.add(Dropout(0.2)) #依一定比例斷開神經元連結，降低模型複雜度，避免過擬合
    model.add(LSTM(units=128))
    model.add(Dropout(0.2))
    model.add(Dense(7, activation='sigmoid')) #模型輸出未來7天的登入機率
    model.compile(loss='binary_crossentropy',
                  optimizer='nadam',
                  metrics=['accuracy'])
    
    return model

model = model0()

In [13]:
history1 = model.fit(train_value, train_y , epochs=10,batch_size=64, verbose=1)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4145 - acc: 0.8221
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.3267 - acc: 0.8813
Epoch 3/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2992 - acc: 0.8981
Epoch 4/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2764 - acc: 0.8997
Epoch 5/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2708 - acc: 0.9010
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2706 - acc: 0.9014
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2712 - acc: 0.9010
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2689 - acc: 0.9011
Epoch 9/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2602 - acc: 0.9043
Epoch 10/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2614 - acc: 0.9020


In [14]:
train_y_pred_lstm = model.predict(train_value) #讓模型輸出未來7天每天的登入機率

In [15]:
train_y_pred_lstm[0]

array([0.47, 0.55, 0.48, 0.48, 0.48, 0.50, 0.48], dtype=float32)

In [16]:
train_y.head(1)

,num_20180401,num_20180402,num_20180403,num_20180404,num_20180405,num_20180406,num_20180407
0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000


In [17]:
a = train_y_pred_lstm[0]
a

array([0.47, 0.55, 0.48, 0.48, 0.48, 0.50, 0.48], dtype=float32)

In [18]:
a = np.round(a) #對機率四捨五入來決定該天是否登入，大於0.5為1(登入)，小於0.5為0(不登入)
a

array([0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00], dtype=float32)

In [19]:
a.sum() #加總7天的登入次數

1.0

In [20]:
pred_df = pd.DataFrame(train_y_pred_lstm) #train_y_pred_lstm為np.array將其轉換成pandas的dataframe

for i in range(7):
    pred_df[i] = np.round(pred_df[i])

pred_df['sum'] = pred_df[0]+pred_df[1]+pred_df[2]+pred_df[3]+pred_df[4]+pred_df[5]+pred_df[6]
pred_df.head()

,0,1,2,3,4,5,6,sum
0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000
2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000
3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000
4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000


In [21]:
result = train_y.copy()
result['遊戲帳號'] = data.account
result['下週登入次數實際值'] = result['num_20180401']+result['num_20180402']+result['num_20180403']+result['num_20180404']+result['num_20180405']+result['num_20180406']+result['num_20180407']

result['下週登入次數預測值']  = pred_df['sum']
result = result[['遊戲帳號', 'num_20180401', 'num_20180402', 'num_20180403', 'num_20180404', 'num_20180405', 'num_20180406', 'num_20180407','下週登入次數實際值', '下週登入次數預測值']]
result.head(20)

,遊戲帳號,num_20180401,num_20180402,num_20180403,num_20180404,num_20180405,num_20180406,num_20180407,下週登入次數實際值,下週登入次數預測值
0,kiss9985,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,1.000000
1,kitming20,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000
2,kitty1100,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000
3,kitty77688,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000
4,kjfgw6305,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000
5,kjlghjbdb,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000
6,kk119gm,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
7,kkaqr033,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,kkboy3001,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000
9,klkjlgc,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000


In [22]:
y_true = result.下週登入次數實際值
y_pred = result.下週登入次數預測值

#用RMSE衡量模型在訓練樣本上的表現
score = np.sqrt(mean_squared_error(y_true, y_pred))
score

1.4866068747318506

## 測試
### 根據前一個月(20180401~20180430)去預測未來7天(20180501~20180507)的登入機率

In [23]:
def model1():
    model = Sequential()
    model.add(LSTM(units=128,input_shape=(test_value.shape[1], test_value.shape[2]), return_sequences=True))#timestep, feature
    model.add(Dropout(0.2)) #依一定比例斷開神經元連結，降低模型複雜度，避免過擬合
    model.add(LSTM(units=128))
    model.add(Dropout(0.2))
    model.add(Dense(7, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='nadam',
                  metrics=['accuracy'])
    
    return model

model1 = model1()

In [24]:
history2 = model1.fit(test_value, test_y , epochs=10,batch_size=64, verbose=1)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4243 - acc: 0.8116
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.3394 - acc: 0.8913
Epoch 3/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2941 - acc: 0.8961
Epoch 4/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2926 - acc: 0.8964
Epoch 5/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2820 - acc: 0.9026
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2760 - acc: 0.9026
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2761 - acc: 0.9043
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2710 - acc: 0.9040
Epoch 9/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2720 - acc: 0.9059
Epoch 10/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.2682 - acc: 0.9046


In [25]:
test_y_pred_lstm = model1.predict(test_value)

In [26]:
ddd = pd.DataFrame(test_y_pred_lstm)

for i in range(7):
    ddd[i] = np.round(ddd[i])

ddd['sum'] = ddd[0]+ddd[1]+ddd[2]+ddd[3]+ddd[4]+ddd[5]+ddd[6]
ddd.head()

,0,1,2,3,4,5,6,sum
0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000
2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000
3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000
4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000


In [27]:
result = test_y.copy()
result['遊戲帳號'] = data.account
result['下週登入次數實際值'] = result['num_20180501']+result['num_20180502']+result['num_20180503']+result['num_20180504']+result['num_20180505']+result['num_20180506']+result['num_20180507']

result['下週登入次數預測值']  = ddd['sum']
result = result[['遊戲帳號', 'num_20180501', 'num_20180502', 'num_20180503', 'num_20180504', 'num_20180505', 'num_20180506', 'num_20180507','下週登入次數實際值', '下週登入次數預測值']]
result.head(200)

,遊戲帳號,num_20180501,num_20180502,num_20180503,num_20180504,num_20180505,num_20180506,num_20180507,下週登入次數實際值,下週登入次數預測值
0,kiss9985,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,4.000000,2.000000
1,kitming20,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000
2,kitty1100,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000
3,kitty77688,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000
4,kjfgw6305,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,7.000000
5,kjlghjbdb,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000
6,kk119gm,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
7,kkaqr033,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,kkboy3001,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000
9,klkjlgc,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000


In [28]:
y_true = result.下週登入次數實際值
y_pred = result.下週登入次數預測值

#用RMSE衡量模型在測試樣本上的表現
score = np.sqrt(mean_squared_error(y_true, y_pred))
score

1.5238110119040353

## 帳號流失月數預測

### 製作帳號每月登入次數表

In [29]:
col = ['accountid', '遊戲名稱', 'time', 'account', 'gamename_plus_time']

In [30]:
data2 = pd.read_table('rawdata/LOGIN_W2_201712_201806.txt', encoding='utf8', names=col)
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3660376 entries, 0 to 3660375
Data columns (total 5 columns):
accountid             int64
遊戲名稱                  object
time                  object
account               object
gamename_plus_time    object
dtypes: int64(1), object(4)
memory usage: 139.6+ MB


In [31]:
data2['登入次數'] = 1
data2['time'] = pd.to_datetime(data2['time'])
data2.head()

,accountid,遊戲名稱,time,account,gamename_plus_time,登入次數
0,21561967,完美世界,2017-12-01,tiamo75941,完美世界2017120100,1
1,21561968,完美世界,2017-12-01,tieoskn,完美世界2017120100,1
2,21561969,完美世界,2017-12-01,tigerpchome5,完美世界2017120100,1
3,21561970,完美世界,2017-12-01,tim8388,完美世界2017120100,1
4,21561971,完美世界,2017-12-01,tk0001,完美世界2017120100,1


In [32]:
df = data2

df['year'] = df.time.dt.year 
df['month'] = df.time.dt.month 
df = df.drop(['遊戲名稱', 'gamename_plus_time', 'accountid'], axis=1)

df.head(1)

,time,account,登入次數,year,month
0,2017-12-01,tiamo75941,1,2017,12


In [33]:
df_201712 =  df[(df['year'] == 2017) & (df['month'] == 12)]
df_201801 =  df[(df['year'] == 2018) & (df['month'] == 1)]
df_201802 =  df[(df['year'] == 2018) & (df['month'] == 2)]
df_201803 =  df[(df['year'] == 2018) & (df['month'] == 3)]
df_201804 =  df[(df['year'] == 2018) & (df['month'] == 4)]
df_201805 =  df[(df['year'] == 2018) & (df['month'] == 5)]

In [34]:
df_201712 = df_201712.groupby(['account'])['登入次數'].sum()
df_201712 = pd.DataFrame(df_201712)
df_201712.rename(columns={'登入次數': 'num_201712'}, inplace=True)

df_201801 = df_201801.groupby(['account'])['登入次數'].sum()
df_201801 = pd.DataFrame(df_201801)
df_201801.rename(columns={'登入次數': 'num_201801'}, inplace=True)

df_201802 = df_201802.groupby(['account'])['登入次數'].sum()
df_201802 = pd.DataFrame(df_201802)
df_201802.rename(columns={'登入次數': 'num_201802'}, inplace=True)

df_201803 = df_201803.groupby(['account'])['登入次數'].sum()
df_201803 = pd.DataFrame(df_201803)
df_201803.rename(columns={'登入次數': 'num_201803'}, inplace=True)

df_201804 = df_201804.groupby(['account'])['登入次數'].sum()
df_201804 = pd.DataFrame(df_201804)
df_201804.rename(columns={'登入次數': 'num_201804'}, inplace=True)



df_201805 = df_201805.groupby(['account'])['登入次數'].sum()
df_201805 = pd.DataFrame(df_201805)
df_201805.rename(columns={'登入次數': 'num_201805'}, inplace=True)

In [35]:
dataframe = [df_201802,df_201803, df_201804, df_201805]

In [37]:
full_data = pd.merge(df_201712, df_201801, left_index=True, right_index=True, how='outer')
for i in dataframe:
    full_data = pd.merge(full_data, i, left_index=True, right_index=True, how='outer')

In [38]:
full_data = full_data.replace(np.nan, 0, regex=True) #regex設為True，它會自動去辨識所有null值格式，避免遺失值沒替換到
full_data.head()

,num_201712,num_201801,num_201802,num_201803,num_201804,num_201805
account,,,,,,
a0000050411,27.000000,8.000000,15.000000,17.000000,11.000000,4.000000
a0000129,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
a00002735,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
a0000330,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
a0000498680,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## 利用Sklearn線性回歸
### 繪製各帳號的回歸線，並依斜率判斷帳號登入次數趨勢是上升還是下降

In [39]:
#創建訓練樣本

def create_dataset(x):
    dataX, dataY = [], []
    for i in range(len(x)):
        a = i
        dataX.append(a)
        dataY.append(x[i])
    return np.array(dataX), np.array(dataY)


In [40]:
x, y = create_dataset(full_data.iloc[0,:])
print(x)
print(y)

[0 1 2 3 4 5]
[27.00 8.00 15.00 17.00 11.00 4.00]


In [41]:
clf = LinearRegression()

In [42]:
coefficient = [] #空白list去儲存回歸線斜率

for i in range(len(full_data)):
    data = full_data.iloc[i,:]
    train_x, train_y = create_dataset(data)
    train_x = train_x.reshape(-1,1)
    
    clf.fit(train_x, train_y)
    coef = clf.coef_  #取得回歸線的斜率
    coefficient.append(coef) #將回歸線的斜率新增至coefficient內


In [43]:
#挑出登入趨勢下降的帳號，也就是回歸線斜率為負的帳號，保存在churn_dataset
churn_dataset = full_data.copy() 
churn_dataset['coef'] = coefficient
churn_dataset = churn_dataset[churn_dataset['coef'] < 0]



In [44]:
churn_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41625 entries, a0000050411 to zzzzzzqq
Data columns (total 7 columns):
num_201712    41625 non-null float64
num_201801    41625 non-null float64
num_201802    41625 non-null float64
num_201803    41625 non-null float64
num_201804    41625 non-null float64
num_201805    41625 non-null float64
coef          41625 non-null object
dtypes: float64(6), object(1)
memory usage: 2.5+ MB


In [45]:
#要預估的月份
month_to_predict = np.array([range(7,24)]) #從第7個月開始直到第24個月
month_to_predict = month_to_predict.reshape(-1,1) #轉換成二維資料

#流失月數
churn_month = []

for i in range(len(churn_dataset.index)):
    data = churn_dataset.iloc[i,:]
    train_x, train_y = create_dataset(data)
    train_x = train_x.reshape(-1,1)
    
    #將train_x轉換為二次多項式
    quadratic_featurizer = PolynomialFeatures(degree=2) #轉換器
    train_x_quadratic = quadratic_featurizer.fit_transform(train_x) #轉換為二次多項式的train_x

    regressor = LinearRegression()
    regressor.fit(train_x_quadratic, train_y) #線性回歸開始訓練
    
    
    test_x_quadratic = quadratic_featurizer.fit_transform(month_to_predict)
    
    trend = regressor.predict(test_x_quadratic) #預測結果儲存在trend變數內
    
    # month為流失月數，預設為0，隨著時間推進1個月，month就加1，直到trend<0時，停止loop
    month = 0
    for j in range(len(trend)):
        if trend[j] > 0:
            month += 1
            if (trend[j] > 0) and (trend[j] < trend[j+1]):
                churn_month.append(month)
                break
        elif trend[j] <= 0:
            churn_month.append(month)
            break

In [46]:
churn_dataset['流失月數預測'] = churn_month
churn_dataset = churn_dataset[['流失月數預測']] #只保留流失月數預測欄位

In [47]:
#合併帳號每月登入次數表和流失月數預測表
churn_df = pd.merge(full_data, churn_dataset,left_index=True, right_index=True, how='outer')
churn_df['遊戲帳號'] = full_data.index
churn_df['登入趨勢'] = coefficient #先前儲存的回歸線斜率

In [48]:
#根據回歸線斜率分類， 3代表上升 2代表持平 1代表下降。
churn_df.loc[(churn_df['登入趨勢'] > 0), '登入趨勢']= 3
churn_df.loc[(churn_df['登入趨勢'] == 0), '登入趨勢']= 2
churn_df.loc[(churn_df['登入趨勢'] < 0), '登入趨勢']= 1

In [49]:
churn_df = churn_df.fillna(churn_df.流失月數預測.max()) #遺失值用最大值填補
churn_df.describe()

,num_201712,num_201801,num_201802,num_201803,num_201804,num_201805,流失月數預測,登入趨勢
count,"73,590.000000","73,590.000000","73,590.000000","73,590.000000","73,590.000000","73,590.000000","73,590.000000","73,590.000000"
mean,8.533605,7.549191,7.624609,8.428577,8.496440,8.562468,3.368093,1.863147
std,12.228076,10.406585,11.122551,12.571595,12.346472,12.383861,3.205837,0.987775
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,18.000000,18.000000,18.000000,20.000000,22.000000,23.000000,7.000000,3.000000
max,31.000000,27.000000,28.000000,31.000000,30.000000,30.000000,7.000000,3.000000


In [50]:
churn_df.head(20)

,num_201712,num_201801,num_201802,num_201803,num_201804,num_201805,流失月數預測,遊戲帳號,登入趨勢
account,,,,,,,,,
a0000050411,27.000000,8.000000,15.000000,17.000000,11.000000,4.000000,0.000000,a0000050411,1
a0000129,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,a0000129,1
a00002735,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,a00002735,1
a0000330,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,a0000330,1
a0000498680,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,a0000498680,1
a000347,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,7.000000,a000347,3
a000529,1.000000,0.000000,3.000000,11.000000,0.000000,1.000000,7.000000,a000529,3
a0005291,0.000000,15.000000,24.000000,16.000000,9.000000,11.000000,7.000000,a0005291,3
a0005292,0.000000,14.000000,24.000000,16.000000,8.000000,11.000000,7.000000,a0005292,3
